In [45]:
import numpy as np
from sklearn import svm
from PIL import Image
from torchvision import transforms
to_tensor = transforms.ToTensor()
import os
import glob
from tqdm import tqdm
from disvae.utils.modelIO import load_model

In [46]:
# Load in the torch vae models
model_name = "btcvae_cardamage_128"
model_dir = os.path.join("results", model_name)
epoch = 0
# model_epoch = "model-{}".format(epoch) #
vae_model = load_model(model_dir) #, filename=model_epoch)


FileNotFoundError: [Errno 2] No such file or directory: 'results/btcvae_cardamage_128/specs.json'

In [35]:
data_root = "data/car_damage_128"
img_paths = []
for ext in ['.jpg', ".JPEG"]:
    img_paths.extend(glob.glob(os.path.join(data_root, "*/*" + ext)))

2035

In [43]:
Z = np.empty(shape=len(img_paths))
y = np.empty(shape=len(img_paths), dtype=int)
for i, img_path in tqdm(enumerate(img_paths)):
    img = to_tensor(Image.open(img_path))
    Z[i] = vae_model.sample_latent(img)
    label = img_path.split('/')[2] # expected path structure: data/<dataset>/<label>/<file>
    label = 0 if label == "Real" else 1
    y[i] = label

0it [00:00, ?it/s]


NameError: name 'vae_model' is not defined

In [13]:
# Train an SVM to classify the latent space between real and fake images from car damage dataset
svm_model = svm.LinearSVC()

# NuSVM hyperparameters
hparams = {
    "nu": .5,  # must be in interval [0,1]
    "kernel": 'linear' # {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
#     degree=3 # only used when kernel='poly'. Degree of the polynomial kernel
}
nusvm_model = svm.NuSVC(**hparams)

In [8]:
nusvm_model.fit(Z, y)